# Tutorial - Text Mining - Classification 

We will predict the category of discussion posts in a newsgroup.

**The unit of analysis is a discussion post**

### Import common packages

In [1]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer 
from nltk import pos_tag, word_tokenize
from sklearn.metrics import accuracy_score
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import SGDClassifier
np.random_seed = 1

In [3]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\priya\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\priya\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [4]:
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\priya\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
nltk.download('omw-1.4')


[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\priya\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [6]:
import os
os.chdir("D:/BAIS/2nd Sem/DSP")

print(os.getcwd())

D:\BAIS\2nd Sem\DSP


### Load data

In [7]:
news = pd.read_csv('news.csv')

news.shape


(597, 5)

In [8]:
news.head(5)

,TEXT,graphics,hockey,medical,newsgroup
0,I have a few reprints left of chapters from my...,1,0,0,graphics
1,"gnuplot, etc. make it easy to plot real valued...",1,0,0,graphics
2,Article-I.D.: snoopy.1pqlhnINN8k1 References: ...,1,0,0,graphics
3,"Hello, I am looking to add voice input capabil...",1,0,0,graphics
4,I recently got a file describing a library of ...,1,0,0,graphics


### Check for missing values

In [9]:
news[['TEXT']].isna().sum()

TEXT    0
dtype: int64

In [10]:
# Define the corpus of documents
corpus = news['TEXT']

transformed_corpus = []
wnl = WordNetLemmatizer()
for document in corpus:
    transformed_document = ""
    for word, tag in pos_tag(word_tokenize(document)):
        wntag = tag[0].lower()
        wntag = wntag if wntag in ['a', 'r', 'n', 'v'] else None
        if not wntag:
            lemma = word
        else:
            lemma = wnl.lemmatize(word, wntag)
        transformed_document+= lemma + " "
    transformed_corpus += [transformed_document]

transformed_corpus

["I have a few reprint leave of chapter from my book `` Visions of the Future '' . These include reprint of 3 chapter probably of interest to reader of this forum , include : 1 . Current Techniques and Development of Computer Art , by Franz Szabo 2 . Forging a Career as a Sculptor from a Career as Computer Programmer , by Stewart Dickson 3 . Fractals and Genetics in the Future by H. Joel Jeffrey I 'd be happy to send out free reprint to researcher for scholarly purpose , until the reprint run out . Just send me your name and address . Thanks , Cliff cliff @ watson.ibm.com ",
 'gnuplot , etc . make it easy to plot real value function of 2 variable but I want to plot function whose value be 2-vectors . I have be do this by plot array of arrow ( complete with arrowhead ) but before go further , I think I would ask whether someone have already do the work . Any pointer ? ? thanx in advance Tom Weston | USENET : weston @ ucssun1.sdsu.edu Department of Philosophy | ( 619 ) 594-6218 ( office 

In [11]:
vectorizer = TfidfVectorizer(stop_words='english', lowercase=True, token_pattern="[^\W\d_]+")

X = vectorizer.fit_transform(transformed_corpus)

df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
df

,aa,aalborg,aamrl,aangeboden,aantal,aaplay,aarnet,ab,abad,abandon,...,zoo,zool,zorn,zt,zu,zubov,zupancic,zurich,zyeh,zzz
0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.09234,0.0,0.0000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.1337,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
592,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
593,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
594,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
595,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Assign the input variable to X and the target variable to y

In [12]:
X = df

This is a multi-class classification problem. There are three categories we will predict:<br>
Whether a post is "graphics," "hockey," or "medical" related

In [13]:
y = news['newsgroup']
y.unique()

array(['graphics', 'hockey', 'medical'], dtype=object)

In [14]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(y)
print(le.classes_)
y = le.transform(y)

y


['graphics' 'hockey' 'medical']


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

## Split the data

In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [16]:
X_train.shape, y_train.shape

((417, 10925), (417,))

In [17]:
X_test.shape, y_test.shape

((180, 10925), (180,))

In [18]:
X_train.head(5)

,aa,aalborg,aamrl,aangeboden,aantal,aaplay,aarnet,ab,abad,abandon,...,zoo,zool,zorn,zt,zu,zubov,zupancic,zurich,zyeh,zzz
586,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
213,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
463,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
327,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
403,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
y_train[:5]

array([2, 1, 2, 1, 2])

Let's create a dataframe to load the model performance metrics into.

In [20]:
performance = pd.DataFrame({"model": [], "n_components": [], "accuracy": []})

## with n_components = 100

## Latent Semantic Analysis (Singular Value Decomposition)

In [21]:
svd = TruncatedSVD(n_components=100, n_iter=10)
X_train_n100= svd.fit_transform(X_train)
X_test_n100 = svd.transform(X_test)

In [22]:
X_train_n100.shape, X_test_n100.shape

((417, 100), (180, 100))

## Random Forest


In [23]:
rnd_clf = RandomForestClassifier(n_estimators=100, max_leaf_nodes=16, n_jobs=-1) 
_ = rnd_clf.fit(X_train_n100, y_train)

### Evaluating Model Performance


In [24]:
#Train accuracy - Not a good measure of model performance as we are using the same data set to train and test
y_pred_train = rnd_clf.predict(X_train_n100)
acc_rf100 = accuracy_score(y_train, y_pred_train)
print(f"Train accuracy with 100_components: {accuracy_score(y_train, y_pred_train):.4f}")

Train accuracy with 100_components: 0.9760


In [25]:
#Test accuracy
y_pred_test = rnd_clf.predict(X_test_n100)
acc_rf100 = accuracy_score(y_test, y_pred_test)
print(f"Test accuracy with 100_components: {accuracy_score(y_test, y_pred_test):.4f}")

Test accuracy with 100_components: 0.8889


In [26]:
performance = pd.concat([performance, pd.DataFrame({'model':"Random Forest", 'n_components': "100", 'accuracy': acc_rf100}, index=[0])])

In [27]:
confusion_matrix(y_test, y_pred_test)

array([[50,  1, 13],
       [ 0, 52,  2],
       [ 3,  1, 58]], dtype=int64)

## Stochastic Gradient Descent Classifier

In [28]:
sgd_clf = SGDClassifier(max_iter=100)
_ = sgd_clf.fit(X_train_n100, y_train)

### Evaluating Model Performance

In [29]:
#Train accuracy
y_pred_train = sgd_clf.predict(X_train_n100)
print(f"Train accuracy with 100_components: {accuracy_score(y_train, y_pred_train):.4f}")

Train accuracy with 100_components: 0.9832


In [30]:
#Test accuracy
y_pred_test = sgd_clf.predict(X_test_n100)
acc_sc100 = accuracy_score(y_test, y_pred_test)
print(f"Test accuracy with 100_components: {accuracy_score(y_test, y_pred_test):.4f}")

Test accuracy with 100_components: 0.9056


In [31]:
performance = pd.concat([performance, pd.DataFrame({'model':"Stochastic Grad Des", 'n_components': "100", 'accuracy':  acc_sc100}, index=[0])])

In [32]:
confusion_matrix(y_test, y_pred_test)

array([[49,  7,  8],
       [ 0, 54,  0],
       [ 2,  0, 60]], dtype=int64)

# n_components = 300


## Latent Semantic Analysis (Singular Value Decomposition)

In [33]:
svd = TruncatedSVD(n_components=300, n_iter=10) #n_components is the number of topics, which should be less than the number of features

X_train_n300= svd.fit_transform(X_train)
X_test_n300 = svd.transform(X_test)


In [34]:
X_train_n300.shape, X_test_n300.shape

((417, 300), (180, 300))

## Random Forest

In [35]:
rnd_clf = RandomForestClassifier(n_estimators=100, max_leaf_nodes=16, n_jobs=-1) 
_ = rnd_clf.fit(X_train_n300, y_train)

### Evaluating Model Performance

In [36]:
#Train accuracy - Not a good measure of model performance as we are using the same data set to train and test
y_pred_train = rnd_clf.predict(X_train_n300)
acc_rf300 = accuracy_score(y_train, y_pred_train)
print(f"Train accuracy with 300_components: {accuracy_score(y_train, y_pred_train):.4f}")

Train accuracy with 300_components: 0.9856


In [37]:
#Test accuracy
y_pred_test = rnd_clf.predict(X_test_n300)
acc_rf300 = accuracy_score(y_test, y_pred_test)
print(f"Test accuracy with 300_components: {accuracy_score(y_test, y_pred_test):.4f}")

Test accuracy with 300_components: 0.8444


In [38]:
performance = pd.concat([performance, pd.DataFrame({'model':"Random Forest", 'n_components': "300", 'accuracy': acc_rf300}, index=[0])])

In [39]:
confusion_matrix(y_test, y_pred_test)

array([[43,  0, 21],
       [ 0, 49,  5],
       [ 1,  1, 60]], dtype=int64)

## Stochastic Gradient Descent Classifier


In [40]:
sgd_clf = SGDClassifier(max_iter=100)
_ = sgd_clf.fit(X_train_n300, y_train)

### Evaluating Model Performance

In [41]:
#Train accuracy
y_pred_train = sgd_clf.predict(X_train_n300)
print(f"Train accuracy with 300_components: {accuracy_score(y_train, y_pred_train):.4f}")

Train accuracy with 300_components: 0.9952


In [42]:
#Test accuracy
y_pred_test = sgd_clf.predict(X_test_n300)
acc_sc300 = accuracy_score(y_test, y_pred_test)
print(f"Test accuracy with 300_components: {accuracy_score(y_test, y_pred_test):.4f}")

Test accuracy with 300_components: 0.9611


In [43]:
performance = pd.concat([performance, pd.DataFrame({'model':"Stochastic Grad Des", 'n_components': "300", 'accuracy': acc_sc300}, index=[0])])

In [44]:
confusion_matrix(y_test, y_pred_test)

array([[60,  2,  2],
       [ 0, 54,  0],
       [ 2,  1, 59]], dtype=int64)

# n_components = 500

## Latent Semantic Analysis (Singular Value Decomposition)


In [45]:
svd = TruncatedSVD(n_components = 500, n_iter=10) #n_components is the number of topics, which should be less than the number of features

X_train_n500= svd.fit_transform(X_train)
X_test_n500 = svd.transform(X_test)

In [46]:
X_train_n500.shape, X_test_n500.shape

((417, 417), (180, 417))

## Random Forest

In [47]:
rnd_clf = RandomForestClassifier(n_estimators=100, max_leaf_nodes=16, n_jobs=-1) 
_ = rnd_clf.fit(X_train_n500, y_train)

### Evaluating Model Performance


In [48]:
#Train accuracy - Not a good measure of model performance as we are using the same data set to train and test
y_pred_train = rnd_clf.predict(X_train_n500)
acc_rf500 = accuracy_score(y_train, y_pred_train)
print(f"Train accuracy with 500_components: {accuracy_score(y_train, y_pred_train):.4f}")

Train accuracy with 500_components: 0.9832


In [49]:
#Test accuracy
y_pred_test = rnd_clf.predict(X_test_n500)
acc_rf500 = accuracy_score(y_test, y_pred_test)
print(f"Test accuracy with 500_components: {accuracy_score(y_test, y_pred_test):.4f}")

Test accuracy with 500_components: 0.8500


In [50]:
performance = pd.concat([performance, pd.DataFrame({'model':"Random Forest", 'n_components': "500", 'accuracy': acc_rf500}, index=[0])])

In [51]:
confusion_matrix(y_test, y_pred_test)

array([[43,  1, 20],
       [ 0, 51,  3],
       [ 2,  1, 59]], dtype=int64)

## Stochastic Gradient Descent Classifier


In [52]:
sgd_clf = SGDClassifier(max_iter=100)
_ = sgd_clf.fit(X_train_n500, y_train)

### Evaluating Model Performance

In [53]:
#Train accuracy
y_pred_train = sgd_clf.predict(X_train_n500)
print(f"Train accuracy with 500_components: {accuracy_score(y_train, y_pred_train):.4f}")

Train accuracy with 500_components: 0.9952


In [54]:
#Test accuracy
y_pred_test = sgd_clf.predict(X_test_n500)
acc_sc500 = accuracy_score(y_test, y_pred_test)
print(f"Test accuracy with 500_components: {accuracy_score(y_test, y_pred_test):.4f}")

Test accuracy with 500_components: 0.9667


In [55]:
performance = pd.concat([performance, pd.DataFrame({'model':"Stochastic Grad Des", 'n_components': "500", 'accuracy': acc_sc500}, index=[0])])

In [56]:
confusion_matrix(y_test, y_pred_test)

array([[61,  1,  2],
       [ 0, 54,  0],
       [ 2,  1, 59]], dtype=int64)

In [57]:
performance

,model,n_components,accuracy
0,Random Forest,100,0.888889
0,Stochastic Grad Des,100,0.905556
0,Random Forest,300,0.844444
0,Stochastic Grad Des,300,0.961111
0,Random Forest,500,0.850000
0,Stochastic Grad Des,500,0.966667


Seeing the performance matrix above, we can say the increasing n_component has increased the accuracy with highest accuracy being 0.966667 for stochastic gradient descent. 
We can see here that increasing the n_component has helped in increasing the accuracy. Accuracy for random forest is .888889, 0.844444 and 0.850000 for n_component 100,300 and 500 respectively. The accuracy for the stochastic gradient descent is 0.905556, 0.961111 and 0.966667 for n_component 100,300 and 500 respectively.
The increasing trend shows that increasing the n_component helps in increasing the making better prediction because it provide the model more data to learn from. 
SVD helps in reducing the data redundancy while keeping the important data which helps in better efficiency and model performance. 